In [1]:
import rockhopper
from rockhopper import loadPLY, exportZA
from rockhopper import testServer
import numpy as np
import os

In [2]:
testServer(os.path.abspath('../sandbox/test.zarr'))

/Users/thiele67/Documents/Python/rock-hopper/rockhopper/ui
 * Serving Flask app 'rockhopper.server'
 * Debug mode: off


 * Running on http://127.0.0.1:3003
Press CTRL+C to quit
127.0.0.1 - - [15/Mar/2025 12:40:03] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [15/Mar/2025 12:40:08] "GET /index.html HTTP/1.1" 200 -
127.0.0.1 - - [15/Mar/2025 12:40:08] "GET /static/js/main.6f1f380c.js HTTP/1.1" 200 -
127.0.0.1 - - [15/Mar/2025 12:40:08] "GET /static/css/main.1d43fd09.css HTTP/1.1" 200 -
127.0.0.1 - - [15/Mar/2025 12:40:08] "GET /manifest.json HTTP/1.1" 200 -
127.0.0.1 - - [15/Mar/2025 12:40:08] "GET /hopper_128.png HTTP/1.1" 200 -
127.0.0.1 - - [15/Mar/2025 12:40:08] "GET /hopper_256.png HTTP/1.1" 200 -


#### Loading data

Lets load an example point cloud. `rockhopper` natively supports PLY format point clouds, which can easily be created/converted using e.g., [CloudCompare](https://www.danielgm.net/cc/).  

In the following we load an example point cloud (you will probably need to change the path) and extract the point coordinates, colours, normals and scalar attributes.

In [2]:
# load data from a PLY file
cloud = loadPLY('../sandbox/testcloud.ply')

# retrieve attributes from resulting dict
xyz = cloud['xyz']
rgb = cloud['rgb']
normals = cloud['normals']
attr = cloud['attr']
attr_names =  cloud['names']

print("Loaded %d points containing %d attributes (%s)"%(len(xyz), len(attr_names), attr_names))

Loaded 15570729 points containing 1 attributes (['scalar_Illuminance_(PCV)'])


#### Preparing for Zarr

Point data is served to our virtual field trip as a chunked array, using a compression and streaming tool called [zarr](https://zarr.readthedocs.io/en/stable/). 

To prepare for this, we need to concatenate our attributes into a single (n,d) array. This will contain all our (n) point coordinates, colours and attributes (d). At a minimum, coordinates and colours must be defined (d=6), but additional bands or properties can be included too (d > 6).

In the following we do this such that for each point we have `x,y,z,r,g,b,i`, where `i` is the scalar field we loaded from our point cloud (see above).

In [3]:
# combine into a single array layout for exporting
xyzrgbattr = np.hstack([xyz, rgb, attr])
print(xyzrgbattr.shape) # everything is concatenated now into a flat array

(15570729, 7)


We also need to define which visualisation options we want available for this point cloud in our virtual field trip. We do this by creating a `style` dictionary that lets us define different true- and false-colour composites to view, and also define a colour-ramp for scalar attribute(s).

In [ ]:
vmn = np.percentile( xyz-np.mean(xyz,axis=0)[None,:], 2, axis=0) # value used as black for our false colour mapping
vmx = np.percentile( xyz-np.mean(xyz,axis=0)[None,:], 98, axis=0) # value used as white for our false colour mapping
stylesheet = { 'rgb':{'R':(3,0,1), # dimensions of our data array to map to "red", vmin, vmax
                 'G':(4,0,1),'B':(5,0,1)}, # other colors (to give true-colour visualisation) 
          'xyz':{'R':(0,vmn[0],vmx[0]), # example false colour visualisation
                 'G':(1,vmn[1],vmx[1]),'B':(2,vmn[2],vmx[2])} }

vmn = np.percentile( xyz[:,2]-np.mean(xyz[:,2]), 2, axis=0) # min value of colour ramp
vmx = np.percentile( xyz[:,2]-np.mean(xyz[:,2]), 98, axis=0) # max value of colour ramp
stylesheet['elev'] = (2, # index of attribute to colour map
                {'scale':'viridis', 'limits':(vmn,vmx,255)}) # colour map properties

vmn = np.percentile( attr[:,0], 2, axis=0) # min value of colour ramp
vmx = np.percentile( attr[:,0], 98, axis=0) # max value of colour ramp
stylesheet['illu'] = (6, {'scale':['#ca0020','#f4a582','#f7f7f7','#92c5de','#0571b0'], # use custom colours!
                      'limits':(vmn,vmx,16)} ) # and use e.g., quantile limits instead of equidistant (e) ones.

In [5]:
stylesheet

{'rgb': {'R': (3, 0, 1), 'G': (4, 0, 1), 'B': (5, 0, 1)},
 'xyz': {'R': (0, -111.18915511475643, 131.60887955321232),
  'G': (1, -142.23502484243363, 116.52297076303512),
  'B': (2, -49.45066771570964, 38.37037720616536)},
 'elev': (2,
  {'scale': 'viridis',
   'limits': (-49.45066771570964, 38.37037720616536, 255)}),
 'illu': (7,
  {'scale': ['#ca0020', '#f4a582', '#f7f7f7', '#92c5de', '#0571b0'],
   'limits': (0.27542373538017273, 0.7245762944221497, 16)})}

### Exporting to zarr

Now we have everything needed to convert our points into zarr format. This is a directory structure that allows the front-end (our web-based virtual fieldtrip viewer) to stream points more efficiently. 

In [6]:
# create our new zarr directory here
out_path = '../sandbox/test.zarr'

In [7]:
exportZA( xyzrgbattr, # our array to export
          out_path, # the path to save this [for uploading to a server later]
          chunk_size=200000, # number of points in each patch streamed to the viewer
          resolution=0.1, # downsample our point cloud to this resolution (important to keep file size low)
          stylesheet=stylesheet) # stylesheet defining how our cloud will be visualised


/Users/thiele67/conda/envs/hylite/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


#### Creating a test server

To test our dataset, we need to launch a local web server that can pass the point data to our web-viewer. For real applications we would upload the datasets to our data storage server (e.g., Google Cloud or Amazon S3), but for testing it is useful to be able to run it locally.